# Association Rules Template

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori,association_rules

## Explore Data

In [ ]:
## Read Data
url = 'file.xlsx'
df = pd.read_excel(url)  #pd.read_csv
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns")

In [ ]:
# visualize the first 5 row of the dataset
df.head()

In [ ]:
df.describe(include='all')

In [ ]:
#n rows with missing values
df.shape[0]-df.dropna().shape[0]

In [ ]:
# Count the number of missing values per columns
df.isna().sum()

In [ ]:
# Count the number of unique values per column
df.nunique()

In [ ]:
# Check the number of rows that contains a specific value

value_to_check = 'value'
value_to_check_int = 0
column_to_scan = 'column_name'
# for string
sum(df[column_to_scan].str.contains(value_to_check))
# for integer
sum(df[column_to_scan] == value_to_check_int)

## Preprocessing

In [ ]:
# remove leading and trailing spaces from string columns
column_to_strip = 'column_name' 
df[column_to_strip] = df[column_to_strip].str.strip()

In [ ]:
# remove all the rows that miss a specific feature (a specific feature is null)
column_that_should_not_be_null = 'example_column'
df1 = df.dropna(axis=0, subset=[column_that_should_not_be_null])

In [ ]:
# Remove all the rows with a specific string
# ~ is Alt+126
string_to_check=''
var_to_check=0
column_to_scan = ''
# for strings
df1 = df[~df[column_to_scan].str.contains(string_to_check)]
# for integer
df1 = df[df[column_to_scan] != var_to_check]

### BASKET CREATION first method: ustack

In [ ]:
# the explanation for this code is below the code

# CREATE THE BASKET
# You're essentially transforming a "long" list of individual transactions 
# into a "wide" matrix where each row is an invoice and each column is a product.

# invoceNo - transaction-id (the same transaction could be split in more rows)
# Description - elemenet of the transaction
# quantity - quantity of the element in the transaction

basket = (df
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo') # in this way, InvoiceNo is not a column anymore
          # .astype(bool) <- you can directly use it instead of the encoder in the next step
          )


basket.head()


# IF THERE IS NO QUANTITY feature :
# use .size() directly

basket = (
    df.groupby(['transaction_id', 'element']).size()
    .unstack()
    .reset_index().fillna(0)
    .set_index('transaction_id')
    # .astype(bool) <- you can directly use it instead of the encoder in the next step
)

#### Basket Creation: Code Explained


### 1. The Original Data (`df`)

This is an exemple DF. Notice how **Invoice 102** appears twice because they bought Apples in two separate batches.

| InvoiceNo | Description | Quantity |
| --- | --- | --- |
| 101 | Apple | 2 |
| 101 | Bread | 1 |
| **102** | **Apple** | **5** |
| **102** | **Apple** | **3** |
| 103 | Bread | 2 |

---

### 2. Group & Sum

`df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum()`

This step collapses the duplicates. It creates a **MultiIndex** (a nested hierarchy).

| InvoiceNo | Description | **Quantity (Sum)** |
| --- | --- | --- |
| **101** | Apple | 2 |
|  | Bread | 1 |
| **102** | Apple | **8** (5+3) |
| **103** | Bread | 2 |

---

### 3. Unstack

`.unstack()`

This is the "Pivot." The `Description` labels (Apple, Bread) jump from being **rows** to being **column headers**.

| InvoiceNo | **Apple** | **Bread** |
| --- | --- | --- |
| **101** | 2.0 | 1.0 |
| **102** | 8.0 | *NaN* |
| **103** | *NaN* | 2.0 |

---

### 4. Reset Index & Fillna

`.reset_index().fillna(0)`

First, we flatten the table (giving it a standard row count 0, 1, 2). Then, we replace the empty `NaN` holes with `0` so the math works later.

|  | InvoiceNo | Apple | Bread |
| --- | --- | --- | --- |
| **0** | 101 | 2.0 | 1.0 |
| **1** | 102 | 8.0 | **0.0** |
| **2** | 103 | **0.0** | 2.0 |

---

### 5. Final Set Index

`.set_index('InvoiceNo')`

This is the final "Basket." The `InvoiceNo` is no longer a data column; it is the **label** for the row. This leaves only the product quantities in the main body of the table.

| **InvoiceNo** (Index) | Apple | Bread |
| --- | --- | --- |
| **101** | 2.0 | 1.0 |
| **102** | 8.0 | 0.0 |
| **103** | 0.0 | 2.0 |


## Basket Creation Second Method: get_dummies

In [ ]:
# THE PROFESSOR NEVER EXPLAINED THIS METHOD IN 2026, BUT I THINCK IT EXPLAINED IT IN PAST YEARS, SO MAYBE CAN BE USEFULL.
df0 = pd.get_dummies(df, prefix='', prefix_sep='', dummy_na=False)
df1 = df0.drop(columns=['element']) # We drop the items column, as we dont need that information. 

# IMPORTANT!!!!
df1 = df1.groupby(level=0, axis=1).sum()

the `get_dummies` approach

Imagine we have this small dataframe (`df`):

| transaction_id | element |
| --- | --- |
| 1 | Milk |
| 1 | Bread |
| 2 | Milk |

---

## Step-by-Step Transformation

### 1. `pd.get_dummies(df, ...)`

This step looks at the `element` column and creates a new column for every unique item it finds.

| transaction_id | Bread | Milk |
| --- | --- | --- |
| 1 | 0 | 1 |
| 1 | 1 | 0 |
| 2 | 0 | 1 |

> **Note:** Notice how Transaction 1 now has two separate rows. This is because the original data had two rows for that ID.

### 2. `df1.groupby(level=0, axis=1).sum()`

The "Magic" happens here. Since we want one row per transaction, we need to collapse (group) the rows. If we group by `transaction_id` and sum the columns, we get our final basket:

| transaction_id | Bread | Milk |
| --- | --- | --- |
| **1** | 1 | 1 |
| **2** | 0 | 1 |

---

### Why use this over Method 1?

In **Method 1**, you used `.unstack()`. Unstacking is like rotating a specific column.
**Method 2** (`get_dummies`) is like exploding the column.

If your data is already "clean" (one item per row), Method 2 is often shorter to write. However, it can be much slower if you have a huge number of unique items (e.g., a supermarket with 50,000 different products), because it creates a massive matrix in your computer's memory before it finishes the grouping step.


## Basket creation third method: oneHotEncoder

In [ ]:
## IN SOME OLD EXAM THE PROFESSOR ASK FOR THIS METHOD. BUT WE NEVER DID IT IN 2026

from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# 1. Setup the Encoder
# sparse_output=False gives us a readable array; handle_unknown ignores new items in future data
encoder = OneHotEncoder(sparse_output=False)

# 2. Fit and Transform the 'element' column
# We reshape to (-1, 1) because the encoder expects a 2D input
encoded_array = encoder.fit_transform(df[['element']])

# 3. Create a DataFrame from the result
# We use encoder.get_feature_names_out() to get the item names back
encoded_df = pd.DataFrame(
    encoded_array, 
    columns=encoder.get_feature_names_out(['element']),
    index=df['transaction_id']
)

# 4. Group by transaction_id to create the basket
basket = encoded_df.groupby('transaction_id').sum()

## Converion Basket To Binary values

In [ ]:
# Remove quantities from basket, we need only the prence of the element in the transaction
def encode_units(x):
    if x >= 1:
        return True
    else:
        return False
    
basket_sets = basket.map(encode_units)

# basket_sets.describe()
# besket_sets.head()

## Rule Creation

In [ ]:
step = 0.01
min_support = 1
min_rules = 20
metric = 'lift'
min_threshold = 1

while True:
    frequent_itemsets = apriori(basket_sets,min_support=min_support,use_colnames=True)
    if frequent_itemsets.shape[0] > 0:
        rules = association_rules(frequent_itemsets,metric=metric,min_threshold=min_threshold)
    if frequent_itemsets.shape[0] > 0 and rules.shape[0] >= min_rules:
        break
    else:
        min_support -= step

print("'min_support'={:4.2f}, number of frequent itemsets={}, number of rules={}"\
      .format(min_support, frequent_itemsets.shape[0], rules.shape[0]))    

In [ ]:
rules

### Display Results

In [ ]:
# Sort the rules by lift and confidence
sorted_rules=rules.sort_values(by=['lift','confidence'],ascending=False).reset_index(drop=True)
sorted_rules.head()

In [ ]:
# Plot a lift vs confidence scatter plot, where the size of the points is proportional to the support of the rule
sorted_rules.plot.scatter(x='lift',y='confidence',s=3**(sorted_rules['support']*30),title='Association Rules')

In [ ]:
# Plot a lift vs confidence scatter plot, where the size of the points is proportional to the support of the rule
s = [1.8**n for n in rules.lift]
rules.plot.scatter(x='support', 
                   y='confidence', 
                   title='Association Rules (dot proportional to Lift)', 
                   s=s)